In [3]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\PARV\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Jumping Jack counter and state
jumping_jack_counter = {'count': 0, 'state': 'closed'}

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmarks = results.pose_landmarks.landmark

        # Detect hand and leg positions
        left_wrist, right_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST], landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
        left_ankle, right_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE], landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

        hand_distance = abs(left_wrist.x - right_wrist.x)
        foot_distance = abs(left_ankle.x - right_ankle.x)

        if hand_distance > 0.4 and foot_distance > 0.4 and jumping_jack_counter['state'] == 'closed':
            jumping_jack_counter['state'] = 'open'
        elif hand_distance < 0.2 and foot_distance < 0.2 and jumping_jack_counter['state'] == 'open':
            jumping_jack_counter['state'] = 'closed'
            jumping_jack_counter['count'] += 1

        # Display count
        cv2.putText(image, f"Jumping Jacks: {jumping_jack_counter['count']}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Jumping Jack Tracker', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
